## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, count, input_file_name, col, desc, sum, when, explode, from_json, to_timestamp, regexp_replace, split, regexp_extract, udf, lit
from transformers import pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import functions as F
from delta.tables import DeltaTable

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# read the source file directory listing
source_file_dir_listings = get_source_listing_df()

In [0]:
# count the number of source files
print(f"There are {len(source_file_dir_listings)} source files")

In [0]:
# examine the head of the data frame (first 5 entries)
print(source_file_dir_listings.head(5))

In [0]:
# print the contents of one of the files
print(show_s3_file_contents("voc_volume/0.json"))

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# define the schema for the raw data
schema = StructType([
    StructField("date", StringType(), True),
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("source_file", StringType(), True),
    StructField("processing_time", TimestampType(), True)
])

In [0]:
# instantiate spark session 
spark = SparkSession.builder.appName("Read/Write Tweet Streams from S3 ALL").getOrCreate()

# read stream
read_stream = (spark.readStream
              .format("cloudFiles")
              .option("cloudFiles.format", "json")
              .option("cloudFiles.inferColumnTypes", "true")
              .option("cloudFiles.schemaEvolutionMode", "failOnNewColumns")
              .schema(schema)
              .load(TWEET_SOURCE_PATH))

# add new columns for processing_time and source_file
read_stream = read_stream.withColumn("processing_time", current_timestamp())
read_stream = read_stream.withColumn("source_file", input_file_name())                               

In [0]:
# setup a write stream using cloudfiles to append to the bronze delta table
bronze_stream = (read_stream
                .writeStream
                .outputMode("append")
                .format("delta")
                .option("mergeSchema", "true") # allow a new schema to be merged into the bronze delta table
                .option("checkpointLocation", BRONZE_CHECKPOINT) # Use the defined BRONZE_CHECKPOINT
                .queryName("bronze_stream")
                .start(BRONZE_DELTA)) # Use the defined BRONZE_DELTA 
    
# awit termination after
bronze_stream.awaitTermination() 


## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# load bronze delta table and examine head
bronze_df = spark.read.format('delta').load(BRONZE_DELTA)
display(bronze_df.head(10))

In [0]:
# Number of tweets captured in bronze table:
print(f"Number of tweets captured in bronze table are: {bronze_df.count()}")

There are no columns that contain Nan or Null values.

In [0]:
# Count the number of tweets by each unique user handle
tweets_by_user = bronze_df.groupBy("user").agg(count("*").alias("tweet_count"))
tweets_by_user.show(10)

In [0]:
# Sort the data by descending count of the number of tweets by each unique user handle
tweet_count_by_user_sorted = bronze_df.groupBy('user') \
    .agg(count('*').alias('tweet_count')) \
    .orderBy(desc('tweet_count'))

bronze_df_sorted = bronze_df \
    .join(tweet_count_by_user_sorted, 'user', 'left_outer') \
    .orderBy(desc('tweet_count')) \
    .drop('tweet_count')


In [0]:
# show tweet count by user sorted
tweet_count_by_user_sorted.show(5)

In [0]:
display(bronze_df_sorted.head(10))

In [0]:
#How many tweets have at least one mention (@) how many tweet have no mentions (@)

# Count the number of tweets with at least one mention (@)
tweets_with_mention = bronze_df_sorted.select(
    sum(when(col("text").rlike("@\\w+"), 1).otherwise(0)).alias("tweets_with_mention")
).collect()[0]["tweets_with_mention"]

# Count the number of tweets with no mentions (@)
tweets_without_mention = bronze_df_sorted.select(
    sum(when(~col("text").rlike("@\\w+"), 1).otherwise(0)).alias("tweets_without_mention")
).collect()[0]["tweets_without_mention"]

# Print the results
print("Number of tweets with at least one mention (@):", tweets_with_mention)
print("Number of tweets with no mentions (@):", tweets_without_mention)

In [0]:
#Plot a bar chart that shows the top 20 tweeters (users)
# Get the top 20 tweeters
top_20_tweeters = tweet_count_by_user_sorted.limit(20)
# Convert to Pandas DataFrame for plotting
top_20_tweeters_pd = top_20_tweeters.toPandas()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(top_20_tweeters_pd['user'], top_20_tweeters_pd['tweet_count'], color='skyblue')
plt.xlabel('User')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [0]:
# write sorted delta table to BRONZE_DELTA 
bronze_df_sorted.write.format("delta").mode("overwrite").save(BRONZE_DELTA)

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Setup read stream
bronze_stream_df = spark.readStream.format("delta").load(BRONZE_DELTA) 

# Convert 'date' column to timestamp
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
bronze_stream_df = bronze_stream_df.withColumn('timestamp', to_timestamp(col('date'), "EEE MMM dd HH:mm:ss zzz yyyy"))

# Extract mentions from 'text' column
mention_pattern = r'@(\w+)'
bronze_stream_df = bronze_stream_df.withColumn('mentions', regexp_extract(col('text'), mention_pattern, 1))

# Explode mentions into separate rows
silver_stream_df = bronze_stream_df.select('timestamp', explode(split(col('mentions'), ' ')).alias('mention'), 
                                           col('text'), col('sentiment'))

# Filter out rows without mentions, since our analysis is only focused on tweets with mentions 
silver_stream_df = silver_stream_df.filter(col('mention') != '')

# Remove mention from 'text' column
silver_stream_df = silver_stream_df.withColumn('cleaned_text', regexp_replace(col('text'), mention_pattern, ''))

# Select required columns for Silver Delta table
silver_stream_df = silver_stream_df.select('timestamp', 'mention', 'cleaned_text', 'sentiment')


In [0]:
# Write stream to Silver Delta table
silver_stream = silver_stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .queryName("silver_stream") \
    .start(SILVER_DELTA)

silver_stream.awaitTermination()

In [0]:
silver_df = spark.read.format("delta").load(SILVER_DELTA)
display(silver_df.head(5))

In [0]:
print(f"Number of tweets captured in the silver table: {silver_df.count()}")

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# load the model and test results on a few texts
model = pipeline(model=HF_MODEL_NAME)

print(f"Result for sentiment extected negative:")
print(model("I am negative to this"))
print(f"Result for sentiment extected positive:")
print(model("I am positive to this"))
print(f"Result for sentiment extected neutral:")
print(model("I am neutral to this"))

In [0]:
# define dictionary to enable conversion from given sentiment label to sentiment id
given_sentiment_label_ids = {'positive': 1, 'negative': 0}
given_sentiment_id_udf = udf(lambda sentiment: given_sentiment_label_ids.get(sentiment), IntegerType())

# define dictionary to enable conversion from predicted sentiment label to sentiment id
# I am putting the neutral predictions in the negative bucket, so assigning it id of 0
predicted_sentiment_label_ids = {'NEG': 0, 'POS': 1, 'NEU': 0} 

def predict_sentiment(text):
    result = model(text)
    label = result[0]['label']
    score = result[0]['score']
    pred_sentiment_id = predicted_sentiment_label_ids[label]
    return label, score, pred_sentiment_id

predict_sentiment_udf = udf(predict_sentiment, StructType([
    StructField('label', StringType()),
    StructField('score', DoubleType()),
    StructField('pred_sentiment_id', IntegerType())
]))

# setup read stream on Silver delta table
silver_delta_df = spark.readStream.format("delta").load(SILVER_DELTA) 

# Apply the UDF to predict sentiment and create new columns
gold_delta_df = silver_delta_df.withColumn('predicted_sentiment_info', predict_sentiment_udf(col('cleaned_text')))

# Make columns from the predicted_sentiment_info column by unpacking
gold_delta_df = gold_delta_df \
    .withColumn('predicted_sentiment', col('predicted_sentiment_info').getField('label')) \
    .withColumn('predicted_score', col('predicted_sentiment_info').getField('score')) \
    .withColumn('predicted_sentiment_id', col('predicted_sentiment_info').getField('pred_sentiment_id'))

# create sentiment_id for given sentiment
gold_delta_df = gold_delta_df.withColumn('sentiment_id', given_sentiment_id_udf(col('sentiment')))


gold_delta_df = gold_delta_df.select('timestamp', 'mention', 'cleaned_text', 'sentiment', 'predicted_score',
                                     'predicted_sentiment', 'sentiment_id', 'predicted_sentiment_id')


In [0]:
# Write stream to Gold Delta table
gold_stream = gold_delta_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .start(GOLD_DELTA)

gold_stream.awaitTermination()

In [0]:
gold_df = spark.read.format("delta").load(GOLD_DELTA)
display(gold_df.head(5))

In [0]:
print(f"Number of tweets captured in the gold table: {gold_df.count()}")

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Extract list of 0s and 1s from predictions and labels in gold delta table
predictions = gold_df.select("predicted_sentiment_id").rdd.flatMap(lambda x: x).collect()
labels = gold_df.select("sentiment_id").rdd.flatMap(lambda x: x).collect()

# Calculate the true positives, false positives and false negatives
TP, FP, FN = 0, 0, 0
for p, l in zip(predictions, labels):
    if p == 1 and l == 1:
        TP += 1
    elif p == 1 and l == 0:
        FP += 1
    elif p == 0 and l == 1:
        FN += 1

# Calculate precision, recall, and F1-score, and confusion matrix
precision = (TP)/(TP + FP)
recall = (TP)/(TP + FN)
f1_score = (2 * precision * recall) / (precision + recall)
conf_matrix = gold_df.crosstab("sentiment_id", "predicted_sentiment_id").toPandas()
mlflow_version = mlflow.__version__
model_name = HF_MODEL_NAME[HF_MODEL_NAME.find('/') + 1:23] + '_model'

# get version of input-silver delta table

sdf = DeltaTable.forPath(spark, SILVER_DELTA)
history_df = sdf.history() \
    .select("version") \
    .orderBy("version", ascending=False)
silver_version = history_df.collect()[0][0]

# Plot the matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix.astype(int), annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.savefig("confusion_matrix.png")

with mlflow.start_run():
    # store the silver delta table version as mlflow parameter
    mlflow.log_param("silver delta_table_version", silver_version)
    # Store the model name and the MLflow version that was used as an MLflow parameters
    mlflow.log_param("model_name", model_name)
    mlflow.log_param("mlflow_version", mlflow_version)
    # Store an image of the confusion matrix as an MLflow artifact
    mlflow.log_artifact("confusion_matrix.png")
    # Store the precision, recall, and F1-score as MLflow metrics
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1_score)

plt.show()

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# count the total number of distinct mentions in the gold delta table
distinct_mention_count = gold_df.select("mention").distinct().count()

print("Total number of distinct mentions int he gold data:", distinct_mention_count)

In [0]:
# make a dataframe for application data by aggregating users with the required information
appdata_df = gold_df.groupBy("mention").agg(
                    F.sum(F.when(F.col("predicted_sentiment") == "NEG", 1).otherwise(0)).alias("negative"),
                    F.sum(F.when(F.col("predicted_sentiment") == "POS", 1).otherwise(0)).alias("positive"),
                    F.sum(F.when(F.col("predicted_sentiment") == "NEU", 1).otherwise(0)).alias("neutral"),
                    F.min("timestamp").alias("min_timestamp"),
                    F.max("timestamp").alias("max_timestamp"))

# Capture the total for each mention in a new column
appdata_df = appdata_df.withColumn("total", F.col("negative") + F.col("positive") + F.col("neutral"))

# Sort the mention count totals in descending order
appdata_df = appdata_df.orderBy(F.col("total").desc())

In [0]:
# Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
appdata_df_pd = appdata_df.toPandas()
top_positive_mentions = appdata_df_pd.sort_values("positive", ascending = False).head(20)
plt.figure(figsize=(10, 6))
plt.bar(top_positive_mentions["mention"], top_positive_mentions["positive"], color='skyblue')
plt.xlabel("Mention (@username)")
plt.ylabel("Positive Count")
plt.title("Top 20 Positive Mentions")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [0]:
# Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)
top_negative_mentions = appdata_df_pd.sort_values("negative", ascending = False).head(20)
plt.figure(figsize=(10, 6))
plt.bar(top_negative_mentions["mention"], top_negative_mentions["negative"], color='skyblue')
plt.xlabel("Mention (@username)")
plt.ylabel("Negative Count")
plt.title("Top 20 Negative Mentions")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# check what streams are running
for stream in spark.streams.active:
    print(stream)

In [0]:
# stop all active streams
stop_all_streams()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
ELAPSED_TIME = END_TIME - START_TIME
print(f"Elapsed time of notebook: {round(ELAPSED_TIME, 2)} seconds")

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE

1. Spill: From the simulator, I did not notice any spill occuring through my pipeline, assuming the dataset size was not as memory intensive to cause a spill.
2. Skew: I did not notice an imbalance in partition size, the dataset, with its balance of negative/positive classes and tasks' mapping to cores, did not seem to cause a skew.
3. Shuffle: There are various wide transforms in the pipeline, like groupBy, aggregate, distinct and joins, which cause shuffles across partitions.
4. Storage: from the data I am able to access in the simulator, the storage format did not seem to be highly efficient given the large number of relatively small json files read and processed.
